In [1]:
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke

backend = FakeSherbrooke()
coupling_map = backend.coupling_map

In [ ]:
import time
import numpy as np
from collections import defaultdict
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit_addon_utils.coloring import auto_color_edges
from pauli_propagation.propagation import propagate_through_circuit

# Num Trotter steps
num_steps = 20

# Set observable
id_pauli = Pauli("I" * backend.num_qubits)
observable = SparsePauliOp(id_pauli.dot(Pauli("Z"), [62]))

# Get a depth-efficient edge coloring
coloring = auto_color_edges(coupling_map)
color_to_edge = defaultdict(list)
for edge, color in coloring.items():
    color_to_edge[color].append(edge)

# Vary rotation angles
hs = [i * np.pi / 32 for i in range(17)]
approx_evs = []
norms = []
times = []
for h in hs:
    circuit = QuantumCircuit(backend.num_qubits)
    for _ in range(num_steps):
        circuit.rx(h, [i for i in range(backend.num_qubits)])
        for edges in color_to_edge.values():
            for edge in edges:
                circuit.rzz(-np.pi / 2, edge[0], edge[1])
    st = time.time()
    propagated_obs, norm = propagate_through_circuit(observable, circuit, 100_000, 1e-12, frame='h')
    times.append(time.time() - st)
    approx_evs.append(float(propagated_obs.coeffs[~propagated_obs.paulis.x.any(axis=1)].sum()))
    norms.append(norm)

In [ ]:
import matplotlib.pyplot as plt

plt.xlabel(r"$R_x$ angle $\theta_h$")
plt.title(r"$\langle Z_{62} \rangle$")
plt.plot(hs, approx_evs, marker='o')